# lg_grade.py

```
랭그래프 사용해서 RAG시스템 구성
벡터DB를 가져와,
질문을 받아 유사한 리트리버를 돌려 1차로 문서를 조회,
문서평가 노드(grade_documents_node)로 2차로 threshold를 기준으로 doc_scores중에 임계치를 넘는 문서만 추리고 해당 문서 반환
조건부 함수에서
1) 조회된 문서가 1개 이상인 경우,  생성노드에 넘어가 프롬프트에 해당 문서들을 넣고,
답변 반환
2) 조회된 문서갯수가 0개(내부문서에 없는 경우)인 경우,
웹검색노드(tvaily 사용)로 넘어가 정보조회 후 답변 

-> retriever 종류별 접근...???
```

```
START
  │
Retrieve Documents (retriever)
  │
Grade Documents
  │
decide_to_generate
 ┌─────────────┐
 │             │
Generate     Web Search
                │
             Generate
                │
               END
```

# lg_rerank.py

```
랭그래프 사용해서 RAG시스템 구성
벡터DB를 가져와,
질문을 받아 유사한 리트리버를 돌려 1차로 문서를 조회,
문서평가 노드(grade_documents_node)로 2차로 threshold를 기준으로 doc_scores중에 임계치를 넘는 문서만 추리고 해당 문서 반환
조건부 함수에서
1) 조회된 문서가 1개 이상인 경우,  생성노드에 넘어가 프롬프트에 해당 문서들을 넣고,
답변 반환
2) 조회된 문서갯수가 0개(내부문서에 없는 경우)인 경우,
웹검색노드(tvaily 사용)로 넘어가 정보조회 후 답변 

웹검색결과
Rerank으로 문서를 정렬하고, generate 생성형llm에게 보내서 답변 반환
```

```
START
  │
Retrieve Documents (Cosine Similarity)
  │
Rerank Documents
  |
Grade Documents
  │
decide_to_generate
 ┌─────────────┐
 │             │
Generate     Web Search
                |
          Rerank Documents
                |
             Generate
                │
               END
```

# lg_hybrid_grade.py

```
기존 lg_hybrid_grade.py 동작 흐름
랭그래프 + hybrid 검색기능(BM25검색 + 벡터검색)을사용해서 RAG시스템 구성

벡터DB를 가져와,
질문을 받아 유사한 리트리버를 돌려 1차로 문서를 랭그래프 + hybrid 검색기능(BM25검색 + 벡터검색)으로 조회,
get_doc_hash_key로 (context : 내용과 source : URL 사용하여)고유 키를 생성하고 고유키를
각 벡터검색 결과와 BM25검색 결과에 매핑하여 저장
각 BM25 검색 결과와 벡터 검색 결과를 하이브리드 스코어링 — RRF (Reciprocal Rank Fusion)으로 누적하여 더한다.
그중에 3개의 문서를 grade_node로 문서를 추린다.

조건부 함수에서
1) 조회된 문서가 1개 이상인 경우,  생성노드에 넘어가 프롬프트에 해당 문서들을 넣고,
답변 반환
2) 조회된 문서갯수가 0개(내부문서에 없는 경우)인 경우,
웹검색노드(tvaily 사용)로 넘어가 정보조회 후 답변을 반환합니다.

web_search 이후에 조회된 문서또한 grade로 문서 추리고나서 generate로 답변 반환

-> web search 이후 grade 노드 추가할지 고민(비교용 랭그래프 시스템이라)
```

```
START
  │
Retrieve (Hybrid: BM25 + Vector)
  │
Grade Documents (threshold-based)
  │
decide_to_generate
 ┌─────────────┐
 │             │
Generate     Web Search
                │
             Generate
                │
               END
```

# lg_hybrid_rerank.py

```

사용자가 질문을 입력한다.
하이브리드 검색(retriever 노드) 실행
	벡터 검색(임베딩 기반) 수행
	BM25 검색(키워드 기반) 수행
문서 내용 + source로 고유 키 생성하여 중복 문서 제거
두 검색 방식에 모두 포함된 문서는 RRF 점수 방식으로 점수 누적
(RRF 공식: 1 / (상수K + rank + 1) 사용)
K=60

RRF 점수를 기준으로 상위 3개의 문서를 후보 문서로 선택 : 1차 문서추리기
문서 재정렬(rerank_documents_node)
후보 문서 3개를 질문과의 관련도 기준으로 재평가
grade로 2차 문서 추리기
조건 분기 (decide_to_generate)
	최종 문서 1개 이상: generate 노드로 이동
	문서 0개: 웹 검색(web_search) 노드로 이동
웹 검색(web_search, Tavily API)
- 내부 DB에서 답을 찾지 못할 경우 웹에서 문서를 수집
Document 구조로 state에 저장
문서 재정렬(rerank_documents_node)후
generate 생성형llm에게 보내서 답변 반환

예: score >= 0.7 인 문서만 유지

기준치를 넘는 문서가 없을 경우 → generate 단계로 가면 할루시네이션 위험이있으므로
그부분은
(일단은) 조회할수없습니다. 로  반환

생성 노드(generate)
	내부 문서 존재 → 내부 문서 기반 답변 생성
	내부 문서 없음 → 웹 검색 문서 기반 답변 생성
최종 생성된 답변이 사용자에게 전달된다.

-> 웹 검색 결과에 대해 relevance score 또는 LLM 평가 점수를 이용하여 필터링 이걸로할지 고민
```

```
START
  │
Retriever (Hybrid: BM25 + Vector)
  │
Rerank Documents
  │
decide_to_generate
 ┌─────────────┐
 │             │
Generate     Web Search
                │
          Rerank Documents
                │
             Generate
                │
               END
```